In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from scipy.ndimage import median_filter as medfilt
import celerite
from celerite import terms
from celerite.modeling import Model
from copy import deepcopy
from scipy.optimize import minimize
from astropy.stats import mad_std
from glob import glob
import emcee
from corner import corner

import sys
sys.path.insert(0, '../')
from libra import kepler296, transit_model, transit_duration

In [5]:
from celerite.solver import LinAlgError

In [7]:
planet = 'e'
paths = sorted(glob('k296_transits/{0}_??.txt'.format(planet)))
planet_params = kepler296(planet)

In [8]:
for i, path in enumerate(paths):
    t, f = np.loadtxt(path, unpack=True)
    # plt.plot(t, f, '.')

    # plt.plot(t, transit_model(t, params), '.')

    # plt.savefig('k296_sc_f.png', bbox_inches='tight')

    t_mean = t.mean()

    class MeanModel(Model):
        parameter_names = ['amp', 't0']

        def get_value(self, t):
            params = deepcopy(planet_params)
            params.t0 = self.t0 + t_mean
            return self.amp * transit_model(t + t_mean, params)

    initp_dict = dict(t0=0, amp=1)
    duration = transit_duration(planet_params)
    parameter_bounds = dict(t0=[-duration, duration],
                            amp=[f.min(),  f.max()])
    mean_model = MeanModel(bounds=parameter_bounds, **initp_dict)

    bounds = dict(log_rho=(-15, 15), log_sigma=(-15, 15))
    kernel = terms.Matern32Term(log_rho=np.log(2), log_sigma=5, bounds=bounds)

    gp = celerite.GP(kernel, mean=mean_model, fit_mean=True)
    gp.compute(t-t.mean(), mad_std(f))

    # plt.plot(t-t_mean, f)
    # plt.plot(t-t_mean, MeanModel(amp=1, t0=0).get_value(t-t_mean))

    # Define a cost function
    def neg_log_like(params, y, gp):
        gp.set_parameter_vector(params)
        return -gp.log_likelihood(y)

    def grad_neg_log_like(params, y, gp):
        gp.set_parameter_vector(params)
        return -gp.grad_log_likelihood(y)[1]
    try: 
        # Fit for the maximum likelihood parameters
        initial_params = gp.get_parameter_vector()
        bounds = gp.get_parameter_bounds()
        soln = minimize(neg_log_like, initial_params, 
                        method="L-BFGS-B", bounds=bounds, args=(f, gp))
        gp.set_parameter_vector(soln.x)
        print("Final log-likelihood: {0}".format(-soln.fun))

        def log_probability(params):
            gp.set_parameter_vector(params)
            lp = gp.log_prior()
            # lp = log_prior(params)
            if not np.isfinite(lp):
                return -np.inf
            return gp.log_likelihood(f) + lp

        initial = np.array(soln.x)
        ndim, nwalkers = len(initial), len(initial) * 4

        sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability,
                                        threads=8) #pool=pool)

        print("Running burn-in...")
        p0 = initial + 1e-3 * np.random.randn(nwalkers, ndim)
        p0, lp, _ = sampler.run_mcmc(p0, 8000)
        sampler.reset()
        p0, lp, _ = sampler.run_mcmc(p0, 1000)

        l, m, u = np.percentile(sampler.flatchain[:, -1], [16, 50, 84])
        np.savetxt("k296_transits/posteriors/{0}_{1:02d}.txt".format(planet, i), np.array([m+t_mean, m-l, u-m]).T)

        corner(sampler.flatchain, label='log_rho log_sigma amp t0'.split())
        plt.savefig('k296_transits/plots/{0}_{1:02d}_corner.png'.format(planet, i), 
                    bbox_inches='tight', dpi=200)
        plt.close()
        n_draws = 20
        plt.figure()
        plt.errorbar(t - t_mean, f, yerr=mad_std(f), fmt='.', color='k', ecolor='gray')
        for j in range(n_draws):
            sample_params = sampler.flatchain[np.random.randint(sampler.flatchain.shape[0]), :]
            mean_model = MeanModel(amp=sample_params[-2], t0=sample_params[-1])
            gp = celerite.GP(kernel, mean=mean_model, fit_mean=True)
            gp.kernel.set_parameter_vector(sample_params[:2])
            gp.compute(t-t.mean(), mad_std(f))

            plt.plot(t-t_mean, gp.mean.get_value(t-t_mean), alpha=0.5, color='DodgerBlue')
        plt.xlabel('Time [d]')
        plt.ylabel('Flux')
        plt.savefig('k296_transits/plots/{0}_{1:02d}_draws.png'.format(planet, i), 
                    bbox_inches='tight', dpi=200)
        plt.close()
    
        del sampler
    except LinAlgError: 
        pass

Final log-likelihood: 385.5793634079298
Running burn-in...


/Users/bmmorris/anaconda/lib/python3.5/site-packages/emcee/ensemble.py:335: RuntimeWarning: invalid value encountered in subtract
  lnpdiff = (self.dim - 1.) * np.log(zz) + newlnprob - lnprob0
/Users/bmmorris/anaconda/lib/python3.5/site-packages/emcee/ensemble.py:336: RuntimeWarning: invalid value encountered in greater
  accept = (lnpdiff > np.log(self._random.rand(len(lnpdiff))))


Final log-likelihood: 272.4442454526357
Running burn-in...
Final log-likelihood: 392.84164731463864
Running burn-in...
Final log-likelihood: 389.71346554697334
Running burn-in...
Final log-likelihood: 396.55984476774586
Running burn-in...
Final log-likelihood: 401.9669548450106
Running burn-in...
Final log-likelihood: 401.9781148368928
Running burn-in...
Final log-likelihood: 152.29982861889795
Running burn-in...
Final log-likelihood: 384.1989284035979
Running burn-in...
Final log-likelihood: 400.92972124653073
Running burn-in...
Final log-likelihood: 401.80682801175874
Running burn-in...
Final log-likelihood: 393.14055243246867
Running burn-in...
Final log-likelihood: 404.53029939908345
Running burn-in...
Final log-likelihood: 399.7446176253519
Running burn-in...
Final log-likelihood: 391.4456327058757
Running burn-in...
Final log-likelihood: 401.15664743444347
Running burn-in...
Final log-likelihood: 8984.57006732158
Running burn-in...
Final log-likelihood: 8497.310573887926
Running 

BlockingIOError: [Errno 35] Resource temporarily unavailable

In [11]:
paths = glob('k296_transits/posteriors/e_*')

np.savetxt('e_posteriors.txt', np.vstack([np.loadtxt(p) for p in paths]))

paths = glob('k296_transits/posteriors/f_*')

np.savetxt('f_posteriors.txt', np.vstack([np.loadtxt(p) for p in paths]))